In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
Table = soup.find('table')
Tabel_split = Table.tbody.text.split('\n')


lst1 =[] ;lst2 =[];lst3 =[] 
temp_val = 0
rows = 1
while (len(Tabel_split)>temp_val):
    lst1.append(Tabel_split[5*rows+1])
    lst2.append(Tabel_split[5*rows+2])
    lst3.append(Tabel_split[5*rows+3])
    temp_val = 5*rows+6
    rows += 1

df=pd.DataFrame(list(zip(lst1,lst2, lst3)), columns =['PostCode', 'Borough', 'Neighborhood']) 
df.head()


df=df[df['Borough']!='Not assigned']
df.head()

df_final = df.groupby(['PostCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#import geocoder # import geocoder

latitude=[]
longitude=[]
print('Looking for coordinats for each postal code ...')
for indx,postcode in enumerate(list(df_final['PostCode'])):
    address = '{} Toronto, Ontario, Canada'.format(postcode)
    print("Progress: {} out of {}".format(indx,df_final['PostCode'].shape[0]-1), end="\r")
    
    try:
        geolocator = Nominatim(user_agent="tn_explorer")
        location = geolocator.geocode(address)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
        
    except:
        latitude.append(None)
        longitude.append(None)
        
print('\nDone!')

df_final['Latitude'] = latitude
df_final['Longitude'] = longitude
print('Number of missing coordinate values is %d which is bascially %.2f%% of all the values.' %(df_final['Latitude'].isna().sum(),(df_final['Latitude'].isna().sum()/df_final['Latitude'].shape[0]*100)))
print()
print("Since we are missing so many values, let's just use the provided csv file")

Looking for coordinats for each postal code ...
Progress: 102 out of 102
Done!
Number of missing coordinate values is 80 which is bascially 77.67% of all the values.

Since we are missing so many values, let's just use the provided csv file


In [2]:
code_lat_long = pd.read_csv('https://cocl.us/Geospatial_data')
code_lat_long.head()

# Droping the incomplete columns for latitude and longitude from previous step
df_final.drop(columns=['Latitude','Longitude'],inplace=True)
# Rename postal code column in the imported dataframe
code_lat_long.rename(columns={'Postal Code':'PostCode'},inplace=True)
# Mergs two dataframe according to the postcode column
df_total=pd.merge(df_final, code_lat_long, on="PostCode")
df_total.head(12)

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [5]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

address = 'Toronto, Ontario'

geolocatorT = Nominatim(user_agent="tn_explorer")
locationT = geolocatorT.geocode(address)
latitudeT = locationT.latitude
longitudeT= locationT.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitudeT, longitudeT))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitudeT, longitudeT], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_total['Latitude'], df_total['Longitude'], df_total['Borough'], df_total['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

print(df_total.shape)
map_toronto

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [6]:
Tronoto_data = df_total[df_total['Borough'].str.contains('Toronto')].reset_index(drop=True)
print('Number of these neighborhoods is ',Tronoto_data.shape[0])
Tronoto_data.head()

Number of these neighborhoods is  38


,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [7]:
# create map of boroughs including the word Toronto
map_Toronto = folium.Map(location=[latitudeT, longitudeT], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Tronoto_data['Latitude'], Tronoto_data['Longitude'], Tronoto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto